In [6]:
import pandas as pd

# Leer el archivo Excel correctamente
df = pd.read_excel(r"C:\Users\nicoa\Desktop\Nueva carpeta\df_completo_original.xlsx")

In [7]:
# Crear la nueva columna "edad²" con los valores al cuadrado
# Renombrar la columna "edad" a "edad2"
df.rename(columns={"ch06": "edad2"}, inplace=True)
df["edad²"] = df["edad2"] ** 2
df["horastrab"] = df["pp3e_tot"] + df["pp3f_tot"]
# Aplicar el coeficiente de ajuste
df["p21"] = df["p21"].astype(float)  # Convertir a float antes de modificar
df.loc[df["ano4"] == 2004, "p21"] *= 314.621568
# Calcular el salario semanal
df["salario_semanal"] = df["p21"] / 40
# Función para calcular años de educación
def calcular_educ(row):
    if row["ch12"] == 2:  # Primario
        return row["ch14"]
    elif row["ch12"] == 4:  # Secundario
        return 12 if row["ch13"] == 1 else row["ch14"]
    elif row["ch12"] == 6:  # Terciario
        return 15 if row["ch13"] == 1 else 12 + row["ch14"]
    elif row["ch12"] == 7:  # Universitario
        return 17 if row["ch13"] == 1 else 12 + row["ch14"]
    elif row["ch12"] == 9:  # Educación especial
        return None  # No comparable
    else:
        return None  # Ns/Nr u otro valor

# Aplicar la función al DataFrame
df["educ"] = df.apply(calcular_educ, axis=1)
df.rename(columns={"ch06": "edad2"}, inplace=True)
# Crear la variable dependiente (y) a partir de estado
df["desocupado"] = (df["estado"] == 2).astype(int)  # 2 = Desocupado, 1 = Ocupado

In [8]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
# Variables predictoras (X) y dependiente (y)
X = df[["edad2", "edad²", "educ", "salario_semanal", "horastrab"]].copy()
y = df["desocupado"]

# Agregar la columna de unos para la regresión
X["constante"] = np.ones(len(df))

# Dividir en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=444)

In [9]:
# Calcular medias en cada grupo
medias_train = X_train.mean()
medias_test = X_test.mean()

# Crear tabla con las diferencias
diferencia_medias = pd.DataFrame({"Train": medias_train, "Test": medias_test})
diferencia_medias["Diferencia"] = diferencia_medias["Train"] - diferencia_medias["Test"]
y_train = y_train.loc[X_train.index]
# Mostrar la tabla
print(diferencia_medias)

                       Train         Test  Diferencia
edad2              33.841266    33.846800   -0.005534
edad²            1635.351659  1634.142179    1.209479
educ               10.087469    10.174397   -0.086928
salario_semanal  2140.781975  2153.207900  -12.425925
horastrab          21.617652    21.959128   -0.341476
constante           1.000000     1.000000    0.000000


In [17]:
import statsmodels.api as sm
datos = pd.concat([X_train, y_train], axis=1)
datos_limpios = datos.replace([np.inf, -np.inf], np.nan).dropna()
X_train_limpio = datos_limpios[X_train.columns]
y_train_limpio = datos_limpios[y_train.name] 
X_train_limpio = sm.add_constant(X_train_limpio)
modelo = sm.OLS(y_train_limpio, X_train_limpio).fit()

# Ver resultados
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:             desocupado   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     233.3
Date:                Tue, 03 Jun 2025   Prob (F-statistic):          6.15e-245
Time:                        00:01:54   Log-Likelihood:                 4264.0
No. Observations:               28336   AIC:                            -8516.
Df Residuals:                   28330   BIC:                            -8467.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
edad2               0.0008      0.000     

In [19]:
print("Dimensión de X_test:", X_test.shape)
print("Dimensión de y_test:", y_test.shape)

Dimensión de X_test: (27402, 6)
Dimensión de y_test: (27402,)


In [23]:
X_test = X_test[X_train.columns.drop('const', errors='ignore')]  # quitar 'const' si se añadió antes
X_test = X_test.replace([np.inf, -np.inf], np.nan).dropna()
y_test = y_test.loc[X_test.index]  # Alinear y_test con X_test limpio
X_test_const = sm.add_constant(X_test)

# Predecir
y_pred = modelo.predict(X_test_const)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (MSE):", mse)

Error cuadrático medio (MSE): 0.04328408367218674


In [39]:
import statsmodels.api as sm
import pandas as pd
from sklearn.impute import SimpleImputer

# Asegurar que 'salario_semanal' no tenga NaN
df = df.dropna(subset=["salario_semanal"])

df["mujer"] = (df["ch04"] == 2).astype(int)

# Definir variables para cada modelo
variables_modelos = [
    ["edad2"],  # Modelo 1
    ["edad2", "edad²"],  # Modelo 2
    ["edad2", "edad²", "educ"],  # Modelo 3
    ["edad2", "edad²", "educ", "mujer"],  # Modelo 4
    ["edad2", "edad²", "educ", "mujer", "horastrab", "desocupado"]  # Modelo 5
]

# Limpiar posibles NaN en columnas
imputer = SimpleImputer(strategy="mean")
columnas_relevantes = list(set(sum(variables_modelos, [])))
df[columnas_relevantes] = imputer.fit_transform(df[columnas_relevantes])

# Variable dependiente
y = df["salario_semanal"]

# Entrenar modelos y guardar resultados
modelos = {}
for i, variables in enumerate(variables_modelos):
    X = df[variables].copy()
    X["constante"] = 1
    modelos[f"Modelo {i+1}"] = sm.OLS(y, X).fit()

# Construir tabla final con coeficientes, desvío y significancia
variables_tabla = ["edad2", "edad²", "educ", "mujer", "horastrab", "desocupado"]
tabla_resultados = pd.DataFrame({"Variable": variables_tabla})

for nombre, modelo in modelos.items():
    fila_modelo = []
    for var in variables_tabla:
        if var in modelo.params:
            coef = modelo.params[var]
            std_err = modelo.bse[var]
            p = modelo.pvalues[var]

            # Significancia
            if p < 0.001:
                stars = "***"
            elif p < 0.05:
                stars = "**"
            elif p < 0.1:
                stars = "*"
            else:
                stars = ""

            # Formato
            fila_modelo.append(f"{coef:.3f} ({std_err:.2f}){stars}")
        else:
            fila_modelo.append("")  # Vacío si la variable no está en ese modelo

    tabla_resultados[nombre] = fila_modelo

# Agregar N y R2
tabla_resultados.loc[len(tabla_resultados)] = ["N"] + [f"{model.nobs:.0f}" for model in modelos.values()]
tabla_resultados.loc[len(tabla_resultados)] = ["R²"] + [f"{model.rsquared:.3f}" for model in modelos.values()]

# Mostrar tabla
pd.set_option('display.max_columns', None)
print(tabla_resultados)
tabla_resultados.to_excel("tabla_modelo_regresion.xlsx", index=False)


     Variable          Modelo 1           Modelo 2           Modelo 3  \
0       edad2  35.386 (0.75)***  268.534 (2.54)***  261.276 (2.56)***   
1       edad²                     -3.004 (0.03)***   -2.936 (0.03)***   
2        educ                                        41.315 (1.92)***   
3       mujer                                                           
4   horastrab                                                           
5  desocupado                                                           
6           N             91339              91339              91339   
7          R²             0.024              0.112              0.117   

               Modelo 4              Modelo 5  
0     261.178 (2.53)***     247.633 (2.58)***  
1      -2.912 (0.03)***      -2.784 (0.03)***  
2      42.884 (1.90)***      40.831 (1.86)***  
3  -1399.775 (31.46)***  -1260.281 (31.11)***  
4                            18.103 (0.45)***  
5                        -2811.831 (75.68)***  
6     